# Import Library


In [35]:
#Import library
! pip install nltk
! pip install sastrawi
! pip install textblob

import csv 
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
nltk.download("vader_lexicon")


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\herog\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [36]:
data = pd.DataFrame()
data

""


# Cleaning


In [37]:
def clean_text(text):
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = re.sub(r'[!$%^&*@#()_+|~=`{}\[\]%\-:";\'<>?,.\/]', ' ', text) 
    text = re.sub(r'[0-9]+','', text)  
    text = re.sub(r'([a-zA-Z])\1\1','\\1', text)
    text = re.sub(r'^[ ]|[ ]$','', text) 
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower() 
    text = re.sub(r'\s+', ' ', text).strip()
    return text
    
data = pd.read_csv("data.csv")
data['Ulasan'] = data['Ulasan'].apply(clean_text)
data.to_csv('data_cleaning.csv', index=False)
data

,Ulasan
0,bangunan sejarah yang keren tempatnya bersih s...
1,banyak spot foto instagrammable di tempat inip...
2,salah satu destinasi wisata yg ada di semarang...
3,icon semarang gedung bersejarah ini dulu kanto...
4,jalan jalan ke semarang jawa tengah belum leng...
...,...
1052,waktu sma sekira tahun waktu itu dipakai oleh ...
1053,arsitektur yang kokoh pada zamanya jika dirawa...
1054,peninggalan ini terasa sekali nuansa klasiknya...
1055,jika dulu orang mendengar nama gedung ini yang...


# Labeling


In [38]:
lexicon_positive = dict()
with open('Lexicon/lexicon_positive.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        lexicon_positive[row[0]] = int(row[1])
lexicon_negative = dict()
with open('Lexicon/lexicon_negative.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        lexicon_negative[row[0]] = int(row[1])
def sentiment_analysis_lexicon_indonesia(text):
    score = 0
    for word_pos in text:
        if (word_pos in lexicon_positive):
            score = score + lexicon_positive[word_pos]
    for word_neg in text:
        if (word_neg in lexicon_negative):
            score = score + lexicon_negative[word_neg]
    polarity = ''
    if (score > 0):
        polarity = 'positif'
    else:
        polarity = 'negatif'
    return score, polarity
data = pd.read_csv('data_cleaning.csv')
data['text'] = data.Ulasan.str.split()
results = data['text'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
data['Sentiment Score'] = results[0]
data['Sentiment'] = results[1]
data.drop(columns=['text'], inplace=True)
data.to_csv('data_labeling.csv', index=False)
print(data['Sentiment'].value_counts())
data

negatif    933
positif    124
Name: Sentiment, dtype: int64


,Ulasan,Sentiment Score,Sentiment
0,bangunan sejarah yang keren tempatnya bersih s...,6,positif
1,banyak spot foto instagrammable di tempat inip...,-16,negatif
2,salah satu destinasi wisata yg ada di semarang...,2,positif
3,icon semarang gedung bersejarah ini dulu kanto...,-8,negatif
4,jalan jalan ke semarang jawa tengah belum leng...,-59,negatif
...,...,...,...
1052,waktu sma sekira tahun waktu itu dipakai oleh ...,-21,negatif
1053,arsitektur yang kokoh pada zamanya jika dirawa...,-1,negatif
1054,peninggalan ini terasa sekali nuansa klasiknya...,-7,negatif
1055,jika dulu orang mendengar nama gedung ini yang...,-16,negatif


# Remove StopWord


In [39]:
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

def remove_stopwords(text):
    return stopword.remove(text)

data = pd.read_csv('data_labeling.csv')
data['Ulasan'] = data['Ulasan'].apply(remove_stopwords)
data.to_csv('data_stopword.csv', index=False)
data

,Ulasan,Sentiment Score,Sentiment
0,bangunan sejarah keren tempatnya bersih suasan...,6,positif
1,banyak spot foto instagrammable tempat inipast...,-16,negatif
2,salah satu destinasi wisata yg semarang yg and...,2,positif
3,icon semarang gedung bersejarah dulu kantor ke...,-8,negatif
4,jalan jalan semarang jawa tengah lengkap belum...,-59,negatif
...,...,...,...
1052,waktu sma sekira tahun waktu dipakai tentara p...,-21,negatif
1053,arsitektur kokoh zamanya dirawat menjadi bangu...,-1,negatif
1054,peninggalan terasa sekali nuansa klasiknya men...,-7,negatif
1055,dulu orang mendengar nama gedung terpikir kean...,-16,negatif


# Normalisasi


In [40]:
slangs = open('Norm/slang.txt', "r", encoding="utf-8", errors='replace')

clear_slangs = []
for newlines in slangs:
    strip_re = newlines.strip("\n")
    split = re.split(r'[:]', strip_re)
    clear_slangs.append(split)
slangs = [[k.strip(), v.strip()] for k, v in clear_slangs]
dict_slangs = {key: values for key, values in slangs}

clean_text = []
for review in data['Ulasan']:
    wordlist = TextBlob(review).words
    for k, v in enumerate(wordlist):
        if v in dict_slangs.keys():
            wordlist[k] = dict_slangs[v]
    clean_text.append(' '.join(wordlist))

data = pd.read_csv('data_stopword.csv')
data['Ulasan'] = clean_text
data.to_csv('data_normalisasi.csv', index=False)
data

,Ulasan,Sentiment Score,Sentiment
0,bangunan sejarah keren tempatnya bersih suasan...,6,positif
1,banyak spot foto instagrammable tempat inipast...,-16,negatif
2,salah satu destinasi wisata yang semarang yang...,2,positif
3,icon semarang gedung bersejarah dahulu kantor ...,-8,negatif
4,jalan jalan semarang jawa tengah lengkap belum...,-59,negatif
...,...,...,...
1052,waktu sama sekira tahun waktu dipakai tentara ...,-21,negatif
1053,arsitektur kokoh zamanya dirawat menjadi bangu...,-1,negatif
1054,peninggalan terasa sekali nuansa klasiknya men...,-7,negatif
1055,dahulu orang mendengar nama gedung terpikir ke...,-16,negatif


# Tokenisasi


In [41]:
data = pd.read_csv('data_normalisasi.csv')
data['Ulasan'] = data['Ulasan'].apply(word_tokenize)
data.to_csv('data_tokenisasi.csv', index=False)
data

,Ulasan,Sentiment Score,Sentiment
0,"[bangunan, sejarah, keren, tempatnya, bersih, ...",6,positif
1,"[banyak, spot, foto, instagrammable, tempat, i...",-16,negatif
2,"[salah, satu, destinasi, wisata, yang, semaran...",2,positif
3,"[icon, semarang, gedung, bersejarah, dahulu, k...",-8,negatif
4,"[jalan, jalan, semarang, jawa, tengah, lengkap...",-59,negatif
...,...,...,...
1052,"[waktu, sama, sekira, tahun, waktu, dipakai, t...",-21,negatif
1053,"[arsitektur, kokoh, zamanya, dirawat, menjadi,...",-1,negatif
1054,"[peninggalan, terasa, sekali, nuansa, klasikny...",-7,negatif
1055,"[dahulu, orang, mendengar, nama, gedung, terpi...",-16,negatif


# Stemming


In [42]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_text(text):
  stem_text = ''.join(text)
  stem = StemmerFactory()
  create_stem = stem.create_stemmer()
  stemmer_stem = create_stem.stem(stem_text)
  return stemmer_stem

data = pd.read_csv('data_tokenisasi.csv')
data['Ulasan'] = data['Ulasan'].apply(stem_text)
data.to_csv('data_bersih.csv', index=False)
print(data['Ulasan'])
data

0       bangun sejarah keren tempat bersih suasana kla...
1       banyak spot foto instagrammable tempat inipast...
2       salah satu destinasi wisata yang semarang yang...
3       icon semarang gedung sejarah dahulu kantor ker...
4       jalan jalan semarang jawa tengah lengkap belum...
                              ...                        
1052    waktu sama sekira tahun waktu pakai tentara pe...
1053    arsitektur kokoh zamanya rawat jadi bangun sej...
1054    tinggal asa sekali nuansa klasik antar imajina...
1055    dahulu orang dengar nama gedung pikir angker d...
1056    kelola lawang sewu obyek wisata budaya sangat ...
Name: Ulasan, Length: 1057, dtype: object


,Ulasan,Sentiment Score,Sentiment
0,bangun sejarah keren tempat bersih suasana kla...,6,positif
1,banyak spot foto instagrammable tempat inipast...,-16,negatif
2,salah satu destinasi wisata yang semarang yang...,2,positif
3,icon semarang gedung sejarah dahulu kantor ker...,-8,negatif
4,jalan jalan semarang jawa tengah lengkap belum...,-59,negatif
...,...,...,...
1052,waktu sama sekira tahun waktu pakai tentara pe...,-21,negatif
1053,arsitektur kokoh zamanya rawat jadi bangun sej...,-1,negatif
1054,tinggal asa sekali nuansa klasik antar imajina...,-7,negatif
1055,dahulu orang dengar nama gedung pikir angker d...,-16,negatif
